# Scraping weather data 

From https://www.wunderground.com/history/daily/

Reference: 
- http://stanford.edu/~mgorkove/cgi-bin/rpython_tutorials/Scraping_a_Webpage_Rendered_by_Javascript_Using_Python.php
- https://automatetheboringstuff.com/chapter11/

In [1]:
import sys
sys.path.append(r'C:\Users\Benny\fastai\old')
from pathlib import Path
from fastai.imports import *
import requests
import wget
from bs4 import BeautifulSoup
from selenium import webdriver 
from datetime import datetime, date
import time

# Mothod 1 

This method has problem with missing data

In [13]:
browser = webdriver.Chrome() 

In [15]:
# pick a date range
start_date = datetime(2016, 3, 18)
stop_date = datetime(2016, 12, 31)
date_range = pd.date_range(start_date, stop_date).strftime('%Y-%m-%d')

In [16]:
len(date_range)

339

In [17]:
weather_df_2016 = pd.DataFrame()

for date in date_range:
    soup = get_soup(date)
    if len(soup.find_all('table'))==0:
        print(f'Cannot obtain data on {date}')
    else: 
        daily_df = get_data_soup(soup)
        #add date column
        daily_df['date'] = pd.to_datetime(date + ' ' +daily_df['Time'], format="%Y-%m-%d %I:%M %p")
        print(f'Finish obtaining data on {date}')
    
    weather_df_2016 = pd.concat([weather_df_2016,daily_df], axis=0, join='outer')
    print(len(weather_df_2016))

weather_df_2016.to_csv('data/weather_16v2.csv')

Finish obtaining data on 2016-01-28
48
Finish obtaining data on 2016-01-29
94
Finish obtaining data on 2016-01-30
142
Finish obtaining data on 2016-01-31
190
Finish obtaining data on 2016-02-01
238
Finish obtaining data on 2016-02-02
286
Finish obtaining data on 2016-02-03
334
Finish obtaining data on 2016-02-04
382
Finish obtaining data on 2016-02-05
430
Finish obtaining data on 2016-02-06
478
Finish obtaining data on 2016-02-07
526
Finish obtaining data on 2016-02-08
574
Finish obtaining data on 2016-02-09
622
Finish obtaining data on 2016-02-10
670
Finish obtaining data on 2016-02-11
718
Finish obtaining data on 2016-02-12
766
Finish obtaining data on 2016-02-13
814
Finish obtaining data on 2016-02-14
862
Finish obtaining data on 2016-02-15
910
Finish obtaining data on 2016-02-16
958
Finish obtaining data on 2016-02-17
1006
Finish obtaining data on 2016-02-18
1054
Finish obtaining data on 2016-02-19
1102
Finish obtaining data on 2016-02-20
1150
Finish obtaining data on 2016-02-21
11

IndexError: list index out of range

In [18]:
weather_df_2016.to_csv('data/weather/weather_16v2.csv')

In [315]:
print(len(weather_df_2016)/48)
weather_df_2016.head()

44.0


,Time,Temperature(F),Dew Point(F),Humidity(%),Wind,Wind Speed(mph),Wind Gust(mph),Pressure(in),Precip.(in),Precip Accum(in),Condition,date
0,12:00 AM,79,64,61,NE,6,0,30.0,0.0,0.0,Fair,2019-01-01 00:00:00
0,12:30 AM,79,64,61,ENE,3,0,30.0,0.0,0.0,Fair,2019-01-01 00:30:00
0,1:00 AM,79,64,61,CALM,0,0,29.9,0.0,0.0,Fair,2019-01-01 01:00:00
0,1:30 AM,79,63,57,CALM,0,0,29.9,0.0,0.0,Fair,2019-01-01 01:30:00
0,2:00 AM,77,63,61,S,3,0,29.9,0.0,0.0,Fair,2019-01-01 02:00:00


In [5]:
# function for parsing html for a single date
def get_soup(date_str):
    url=f'https://www.wunderground.com/history/daily/th/bang-phut/VTBD/date/{date_str}'
    browser.get(url)
    time.sleep(10)
    innerhtml= browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerhtml)
    return soup

In [6]:
# extract daily_df from html table return daily_df 
def get_data_soup(soup):
    div_table=soup.find_all(attrs={"id": "history-observation-table"})[0]
    head=[ tag.string for tag in div_table.thead.find_all('button')]
    trow = div_table.tbody.find_all('tr')
    daily_df=pd.DataFrame()
    #stripping unit
    f_row = trow[0]
    units=[]
    for unit_cell in f_row.find_all(class_="wu-label"):
        unit = ''.join(string for string in unit_cell.stripped_strings)
        units.append(r'('+unit+r')')
    
    units.insert(0, '')
    units.insert(4, '')
    units.append('')
    # stripping data 
    for row in trow:
        row_data = get_row_data(row)
        daily_df=daily_df.append(pd.DataFrame(row_data).T) 
        
    head_unit = [a+b for a, b in zip(head,units)]
    daily_df.columns = head_unit
    return daily_df


In [7]:
# extract row data 
def get_row_data(row):
    cells = row.find_all('ng-saw-cell-parser')
    row_data=[]
    for cell in cells:
        cell_data = [string for string in cell.stripped_strings]
        row_data.append(cell_data[0])
    return row_data

# Method 2
1. obtain the soup object of the table and parse it as daily dataframe
2. use soup object to find the unit
3. strip the unit out
4. append daily df to the weather df 

## Testing section

In [ ]:
browser = webdriver.Chrome() 

In [55]:
start_date = datetime(2016, 11, 21)
stop_date = datetime(2016, 12, 31)
date_range = pd.date_range(start_date, stop_date).strftime('%Y-%m-%d')
date_str = date_range[0]

In [56]:
url=f'thtps://www.wunderground.com/history/daily/th/bang-phut/VTBD/date/{date_str}'
browser.get(url)
time.sleep(10)
innerhtml= browser.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerhtml)
div_table=soup.find_all(attrs={"id": "history-observation-table"})[0]
daily_df = pd.read_html(str(div_table))[0]

In [57]:
daily_df = df[0]
daily_df.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Precip Accum,Condition
0,12:00 AM,84,79,84,SSW,10,0,29.8,0.0,0.0,Fair
1,12:30 AM,82,79,89,SSW,9,0,29.8,0.0,0.0,Fair
2,1:00 AM,82,79,89,SSW,8,0,29.8,0.0,0.0,Fair
3,1:30 AM,82,81,94,S,8,0,29.8,0.0,0.0,Fair
4,2:00 AM,82,79,89,S,8,0,29.8,0.0,0.0,Fair


In [58]:
# add unit into header

trow = div_table.tbody.find_all('tr')

 #stripping unit
f_row = trow[0]
units=[]
for unit_cell in f_row.find_all(class_="wu-label"):
    unit = ''.join(string for string in unit_cell.stripped_strings)
    units.append(r'('+unit+r')')
units.insert(0, '')
units.insert(4, '')
units.append('')

In [59]:
daily_df.columns + units

Index(['Time', 'Temperature(F)', 'Dew Point(F)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Precip Accum(in)', 'Condition'],
      dtype='object')

In [40]:
daily_df.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Precip Accum,Condition
0,12:00 AM,84,79,84,SSW,10,0,29.8,0.0,0.0,Fair
1,12:30 AM,82,79,89,SSW,9,0,29.8,0.0,0.0,Fair
2,1:00 AM,82,79,89,SSW,8,0,29.8,0.0,0.0,Fair
3,1:30 AM,82,81,94,S,8,0,29.8,0.0,0.0,Fair
4,2:00 AM,82,79,89,S,8,0,29.8,0.0,0.0,Fair


In [39]:
# clean up data
daily_df.iloc[:,1] = daily_df.iloc[:,1].str.replace(' F','')
daily_df.iloc[:,2] = daily_df.iloc[:,2].str.replace(' F','')
daily_df.iloc[:,3] = daily_df.iloc[:,3].str.replace(' %','')
daily_df.iloc[:,5] = daily_df.iloc[:,5].str.replace(' mph','')
daily_df.iloc[:,6] = daily_df.iloc[:,6].str.replace(' mph','')
daily_df.iloc[:,7] = daily_df.iloc[:,7].str.replace(' in','')
daily_df.iloc[:,8] = daily_df.iloc[:,8].str.replace(' in','')
daily_df.iloc[:,9] = daily_df.iloc[:,9].str.replace(' in','')

In [41]:
weather = pd.DataFrame()
weather = pd.concat([weather,daily_df])

In [42]:
weather.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Precip Accum,Condition
0,12:00 AM,84,79,84,SSW,10,0,29.8,0.0,0.0,Fair
1,12:30 AM,82,79,89,SSW,9,0,29.8,0.0,0.0,Fair
2,1:00 AM,82,79,89,SSW,8,0,29.8,0.0,0.0,Fair
3,1:30 AM,82,81,94,S,8,0,29.8,0.0,0.0,Fair
4,2:00 AM,82,79,89,S,8,0,29.8,0.0,0.0,Fair


## Method 2 Action 

In [2]:
import sys
sys.path.append(r'C:\Users\Benny\fastai\old')
from pathlib import Path
from fastai.imports import *
import requests
import wget
from bs4 import BeautifulSoup
from selenium import webdriver 
from datetime import datetime, date
import time

In [10]:
browser = webdriver.Chrome() 

In [14]:
start_date = datetime(2016, 12, 21)
stop_date = datetime(2016, 12, 31)
date_range = pd.date_range(start_date, stop_date).strftime('%Y-%m-%d')

In [15]:
weather_2016 = pd.DataFrame()

for date in date_range:
    daily_df, div_table = get_data_n_soup(date)
    units = get_unit(div_table)
    daily_df.columns = daily_df.columns+units
    
    print(f'Finish obtaining data on {date}')
    
    weather_2016 = pd.concat([weather_2016,daily_df], axis=0, join='outer')
    print(len(weather_2016))
    
#save weather dataframe
weather_2016.to_csv('data/weather/weather_2016v2.csv')


Finish obtaining data on 2016-12-21
48
Finish obtaining data on 2016-12-22
96
Finish obtaining data on 2016-12-23
144
Finish obtaining data on 2016-12-24
192
Finish obtaining data on 2016-12-25
240
Finish obtaining data on 2016-12-26
287
Finish obtaining data on 2016-12-27
335
Finish obtaining data on 2016-12-28
383
Finish obtaining data on 2016-12-29
430
Finish obtaining data on 2016-12-30
478
Finish obtaining data on 2016-12-31
526


In [13]:
weather_2016.to_csv('data/weather/weather_2016v1.csv')

In [3]:
def get_data_n_soup(date_str):
    url=f'https://www.wunderground.com/history/daily/th/bang-phut/VTBD/date/{date_str}'
    browser.get(url)
    time.sleep(10)
    innerhtml= browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerhtml)
    div_table=soup.find_all(attrs={"id": "history-observation-table"})[0]
    daily_df = pd.read_html(str(div_table))[0]
    
    # clean up data
    daily_df.iloc[:,1] = daily_df.iloc[:,1].str.replace(' F','')
    daily_df.iloc[:,2] = daily_df.iloc[:,2].str.replace(' F','')
    daily_df.iloc[:,3] = daily_df.iloc[:,3].str.replace(' %','')
    daily_df.iloc[:,5] = daily_df.iloc[:,5].str.replace(' mph','')
    daily_df.iloc[:,6] = daily_df.iloc[:,6].str.replace(' mph','')
    daily_df.iloc[:,7] = daily_df.iloc[:,7].str.replace(' in','')
    daily_df.iloc[:,8] = daily_df.iloc[:,8].str.replace(' in','')
    daily_df.iloc[:,9] = daily_df.iloc[:,9].str.replace(' in','')
    
    # add date columns
    daily_df['date'] = pd.to_datetime(date + ' ' +daily_df['Time'], format="%Y-%m-%d %I:%M %p")
    return daily_df, div_table

In [4]:
def get_unit(div_table):
    trow = div_table.tbody.find_all('tr')
    #stripping unit
    f_row = trow[0]
    units=[]
    for unit_cell in f_row.find_all(class_="wu-label"):
        unit = ''.join(string for string in unit_cell.stripped_strings)
        units.append(r'('+unit+r')')
    units.insert(0, '')
    units.insert(4, '')
    units.append('')
    units.append('')
    return units